## R & R

In [121]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from collections import Counter
laptop = True
import pickle

import logging
logging.basicConfig(filename='ML.log',level=logging.DEBUG)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from ml_functions import feature_creation, conf_eval, data_prep, test_model

from functions import count_mode, bayes_probs, bayes_binary

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

#crowd_size, feature_transform
from path import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Importing Data

In [122]:
test_data = pd.read_csv(prepared_data + "test_data_large.csv")
test_data_covid = pd.read_csv(prepared_data + "test_data_large_covid.csv")
test_data_noncovid = pd.read_csv(prepared_data + "test_data_large_noncovid.csv")

test_data_large = test_data.append(test_data_covid)
test_data_large = test_data_large.append(test_data_noncovid)

test_data_large.reset_index(inplace = True, drop = True)
test_target_large = test_data_large['mode'] == 'FM'


X_test = feature_creation(test_data_large)


In [123]:
1 - test_target_large.sum()/test_target_large.shape[0]

0.5387857142857143

In [124]:
train_data = pd.read_csv(prepared_data + "train_data_large.csv")
val_data = pd.read_csv(prepared_data + "val_data_large.csv")

train_data_covid = pd.read_csv(prepared_data + "train_data_large_covid.csv")
val_data_covid = pd.read_csv(prepared_data + "val_data_large_covid.csv")


train_data_noncovid = pd.read_csv(prepared_data + "train_data_large_noncovid.csv")
val_data_noncovid = pd.read_csv(prepared_data + "val_data_large_noncovid.csv")



train_data_large = train_data.append(train_data_covid)
train_data_large = train_data_large.append(train_data_noncovid)

train_data_large.reset_index(inplace = True, drop = True)

val_data_large = val_data.append(val_data_covid)
val_data_large = val_data_large.append(val_data_noncovid)

val_data_large.reset_index(inplace = True, drop = True)


train_target_large = train_data_large['mode'] == 'FM'
val_target_large = val_data_large['mode'] == 'FM'

X_train = feature_creation(train_data_large)
X_val = feature_creation(val_data_large)


In [137]:
1 - (test_target_large.sum()+train_target_large.sum() + val_target_large.sum()) /(test_target_large.shape[0]+train_target_large.shape[0] + val_target_large.shape[0])                                                                                                                          

0.5795407407407407

In [138]:
1 - (test_target_large.sum()) /(test_target_large.shape[0])


0.5387857142857143

### Test Sets

In [127]:
test_set = pickle.load(open(data_pickles + 'test_set.p', "rb" ))
test_set_covid = pickle.load(open(data_pickles + 'test_set_covid.p', "rb" ))
test_set_noncovid = pickle.load(open(data_pickles + 'test_set_noncovid.p', "rb" ))

test_set_all = test_set + test_set_covid + test_set_noncovid
test_set_non_covid_all = test_set + test_set_noncovid
test_set_two = test_set_covid + test_set_noncovid

In [128]:
len(test_set_all)

28

In [130]:
len(list(test_data_large.article_num.unique()))

28

In [91]:
len(test_set)

19

In [19]:
len(test_set_covid)

5

In [81]:
len(test_set_noncovid)

4

In [18]:
source_dic = {}

for an_art in test_set:
    
    source_dic[an_art] = 0 
    
for an_art in test_set_covid:
    
    source_dic[an_art] = 1 
    
for an_art in test_set_noncovid:
    
    source_dic[an_art] = 2 

In [43]:
article_dic_covid = pickle.load(open(data_pickles + 'article_dic_covid.p', "rb" ))
article_dic_noncovid = pickle.load(open(data_pickles + 'article_dic_noncovid.p', "rb" ))
article_dic = pickle.load(open(data_pickles + 'article_dic.p', "rb" ))

all_dic = {}
all_dic.update(article_dic)
all_dic.update(article_dic_covid)
all_dic.update(article_dic_noncovid)

In [69]:
all_dic[12]

('FM', 4.0, 'nov 18', 2)

In [42]:
for an_art in test_set_covid:
    
    print(article_dic_covid[an_art])

('FM', 4.2, 'Jun 15', 3)
('FM', 2.0, 'Jun 1', 3)
('T', 5.6, 'Jun 15', 2)
('T', 6.5, 'May 28', 4)
('T', 4.5, 'Jun 3', 4)


## Missing articles

In [126]:
all_arts_set = list(test_data_large.article_num.unique()) + list(train_data_large.article_num.unique()) + list(val_data_large.article_num.unique())
len(all_arts_set)

135

In [44]:
org_test = pickle.load(open(data_pickles + "test_set.p", "rb"))
org_val = pickle.load(open(data_pickles + "val_set.p", "rb"))
org_train = pickle.load(open(data_pickles + "train_set.p", "rb"))
org_test_non = pickle.load(open(data_pickles + "test_set_noncovid.p", "rb"))
org_val_non = pickle.load(open(data_pickles + "val_set_noncovid.p", "rb"))
org_train_non = pickle.load(open(data_pickles + "train_set_noncovid.p", "rb"))
org_test_cov = pickle.load(open(data_pickles + "test_set_covid.p", "rb"))
org_val_cov = pickle.load(open(data_pickles + "val_set_covid.p", "rb"))
org_train_cov = pickle.load(open(data_pickles + "train_set_covid.p", "rb"))

all_sets_list_full = [org_test, org_val, org_train, org_test_non, org_val_non, org_train_non, org_test_cov, org_val_cov, org_train_cov]
all_sets_list =  [item for sublist in all_sets_list_full for item in sublist]

In [67]:
### Check to see if everything in actual data matches what should be in there according to sets

for an_art in all_arts_set:
    
    if an_art not in all_sets_list:
        
        print(and_art, "mistake")

In [68]:
### checking to see that all articles are of correct type

In [73]:
### checking to see that all articles are of correct type
for an_art in all_arts_set:
    
    if all_dic[an_art][3] > 3:
        
        print(an_art, all_dic[an_art], "mistake")

In [77]:
### Checking fact checker standard 
article_all_covid = pickle.load(open(data_pickles + 'article_all_covid.p', "rb" ))
article_all_noncovid = pickle.load(open(data_pickles + 'article_all_noncovid.p', "rb" ))
article_all = pickle.load(open(data_pickles + 'article_all.p', "rb" ))

all_fact_articles = []
#all low credibility
votes_list_lowcred = [x for x, y in article_all.items() if 0 < x % 5 < 4 and x > 10]
votes_list_lowcred_covid = [x for x, y in article_all_covid.items() if 0 < x % 5 < 4]
votes_list_lowcred_noncovid = [x for x, y in article_all_noncovid.items() if 0 < x % 5 < 4]

all_fact_articles.extend(votes_list_lowcred)
all_fact_articles.extend(votes_list_lowcred_covid)
all_fact_articles.extend(votes_list_lowcred_noncovid)


In [79]:
### Check to see if everything in fact checker standards matches what should be in there according to sets

for an_art in all_fact_articles:
    
    if an_art not in all_sets_list:
        
        print(and_art, "mistake")

In [98]:
full_size = 0
combine_list = []
for a_set in all_sets_list:
    
    full_size += len(a_set)
    combine_list = combine_list + a_set
    
print(full_size)

135


In [100]:
for a_set in combine_list:
    
    if a_set not in all_arts_set:
        
        print(a_set)

201
228
208


In [101]:
article_dic_covid[201]

('FM', 2.67, 'May 28', 2)

In [102]:
article_dic_covid[208]

('T', 6.0, 'Jun 1', 4)

In [103]:
article_dic_covid[228]

('T', 5.6, 'Jun 15', 4)

In [106]:
201 in org_val_cov

True

In [107]:
208 in org_val_cov

True

In [109]:
228 in org_val_cov

True

In [113]:
org_val_cov

[201, 228, 208]

In [112]:
val_data_covid.article_num.unique()

array([201, 228, 208])

## Info on overall ratings

In [131]:
all_arts_set = list(test_data_large.article_num.unique()) + list(train_data_large.article_num.unique()) + list(val_data_large.article_num.unique())

In [132]:
type_list = []

for a_key in all_dic.keys():
    
    if a_key in all_arts_set:
        
        type_list.append(all_dic[a_key][0])

In [133]:
rate_dist = Counter(type_list)
rate_dist

Counter({'FM': 57, 'T': 58, 'No Mode!': 16, 'CND': 4})

In [85]:
len(test_set)

19

In [135]:
fm_counter = 0

for an_art in all_arts_set:

    info = all_dic[an_art]
    
    if info[0] == "FM":
        
        fm_counter += 1
        
        


In [136]:
fm_counter

57

### NN with 25

In [99]:
X_test_large = feature_creation(test_data_large, crowd_num = 25)
X_train_large = feature_creation(train_data_large, crowd_num = 25)

In [100]:
numeric_features_large = []

resp_var = [x for x in X_test_large.columns if 'resp_veracity_' in x]
new_cols = [x for x in X_test_large.columns if 'new' in x]

#numeric_features.extend(resp_var)
numeric_features_large.extend(resp_var) #rempve this if it doesn't work
numeric_features_large.extend(new_cols)
numeric_features_large.extend(["crowd_means",'crowd_median','crowd_full_range', 'crowd_IQR_range', \
                         'crowd_variance', 'crowd_bayes'])

In [101]:
#nn = pickle.load(open(local_pickles + 'model.p', "rb" ))



class Net(nn.Module):

    def __init__(self, emb_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(emb_dim, 500) 
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 25)
        self.fc4 = nn.Linear(25, 25)
        self.fc5 = nn.Linear(25, 10)
        self.fc6 = nn.Linear(10, 1)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        
        return x
    
    
class Net_dropout(nn.Module):

    def __init__(self, emb_dim):
        super(Net_dropout, self).__init__()
        self.fc1 = nn.Linear(emb_dim, 500) 
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 25)
        self.fc4 = nn.Linear(25, 25)
        self.fc5 = nn.Linear(25, 10)
        self.fc6 = nn.Linear(10, 1)
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)     
        x = F.relu(self.fc3(x))    
        x = self.dropout(x)   
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        
        return x
    



In [102]:

emb_dim = 121   
net_large = Net(emb_dim)
net_large.load_state_dict(torch.load(models + "model_large.pt"))
net_large.eval()

nn_list_large = ["nn_25"]

# Preparing the data
y_data_bin_large = np.where(test_target_large == True, 1, 0)


test = StandardScaler()
test.fit(X_train_large[numeric_features_large])

#training data
X_train_stand_large = X_test_large.copy()
X_train_stand_large[numeric_features_large] = test.transform(X_test_large[numeric_features_large])

train_dat_nn_large = data_prep(X_train_stand_large,test_target_large)

nn_preds  = torch.sigmoid(net_large(torch.tensor(train_dat_nn_large[:][0]).float()))
nn_preds = np.where(nn_preds > .5, 1,0).reshape(-1,)

all_results = list(conf_eval(nn_preds, test_target_large))

nn_list_large.extend(all_results)

#performance_dic['nn_25'] = nn_list_large



In [103]:
#performance on covid set
list(conf_eval(nn_preds[test_data_large.article_num.isin(test_set_covid)], test_target_large[test_data_large.article_num.isin(test_set_covid)]))

[0.6844,
 0.5727762803234502,
 0.4272237196765499,
 0.8474409448818898,
 0.15255905511811024]

In [104]:
#performance on noncovid set
list(conf_eval(nn_preds[test_data_large.article_num.isin(test_set_non_covid_all)], test_target_large[test_data_large.article_num.isin(test_set_non_covid_all)]))

[0.680695652173913,
 0.4651116026543334,
 0.5348883973456666,
 0.8449517389305959,
 0.155048261069404]

In [105]:
#performance on period one
list(conf_eval(nn_preds[test_data_large.article_num.isin(test_set)], test_target_large[test_data_large.article_num.isin(test_set)]))

[0.6942105263157895,
 0.5161127895266868,
 0.4838872104733132,
 0.822178002894356,
 0.177821997105644]

In [106]:
#performance on perido two
list(conf_eval(nn_preds[test_data_large.article_num.isin(test_set_two)], test_target_large[test_data_large.article_num.isin(test_set_two)]))

[0.6542222222222223,
 0.447887323943662,
 0.5521126760563381,
 0.9086848635235732,
 0.0913151364764268]

## NN with high PK 

In [107]:
numeric_features = []

resp_var = [x for x in X_train.columns if 'resp_veracity_' in x]
new_cols = [x for x in X_train.columns if 'new' in x]

#numeric_features.extend(resp_var)
numeric_features.extend(resp_var) #rempve this if it doesn't work
numeric_features.extend(new_cols)
numeric_features.extend(["crowd_means",'crowd_median','crowd_full_range', 'crowd_IQR_range', \
                         'crowd_variance', 'crowd_bayes'])


In [108]:
pol_knowledge_df_test = pickle.load(open(data_pickles + 'pol_knowledge_df_test.p', "rb" ))
X_data_pol_test = feature_creation(pol_knowledge_df_test)

In [109]:
pol_knowledge_df_train_orig = pickle.load(open(data_pickles + 'pol_knowledge_df_train_orig.p', "rb" ))
# Preparing the data
X_data_pol_train = feature_creation(pol_knowledge_df_train_orig)

In [110]:

emb_dim = 61 
net_large = Net(emb_dim)
net_large.load_state_dict(torch.load(models + "model_nn_highpol.pt"))
net_large.eval()

nn_list_large = ["nn_10_highpol"]

# Preparing the data
y_data_bin_large = np.where(test_target_large == True, 1, 0)


test = StandardScaler()
test.fit(X_data_pol_train[numeric_features])

#training data
X_test_stand_large = X_data_pol_test.copy()
X_test_stand_large[numeric_features] = test.transform(X_data_pol_test[numeric_features])


test_dat_nn_large = data_prep(X_test_stand_large,y_data_bin_large)


nn_preds  = torch.sigmoid(net_large(torch.tensor(test_dat_nn_large[:][0]).float()))
nn_preds_high_pk  = np.where(nn_preds > .5, 1,0).reshape(-1,)


all_results = list(conf_eval(nn_preds_high_pk, test_target_large))

#performance_dic['nn_10_highpol'] = nn_list_large


In [111]:
#performance on covid set
list(conf_eval(nn_preds_high_pk[test_data_large.article_num.isin(test_set_covid)], test_target_large[test_data_large.article_num.isin(test_set_covid)]))

[0.6756,
 0.648921832884097,
 0.35107816711590295,
 0.7145669291338582,
 0.2854330708661417]

In [112]:
#performance on noncovid set
list(conf_eval(nn_preds_high_pk[test_data_large.article_num.isin(test_set_non_covid_all)], test_target_large[test_data_large.article_num.isin(test_set_non_covid_all)]))

[0.6828695652173913,
 0.5274482203901065,
 0.4725517796098934,
 0.8012869618507737,
 0.19871303814922628]

In [113]:
#performance on period one
list(conf_eval(nn_preds_high_pk[test_data_large.article_num.isin(test_set)], test_target_large[test_data_large.article_num.isin(test_set)]))

[0.6992631578947368,
 0.5934038267875126,
 0.4065961732124874,
 0.7753256150506512,
 0.22467438494934877]

In [114]:
#performance on period two 
list(conf_eval(nn_preds_high_pk[test_data_large.article_num.isin(test_set_two)], test_target_large[test_data_large.article_num.isin(test_set_two)]))

[0.6442222222222223,
 0.49456740442655933,
 0.5054325955734407,
 0.8287841191066998,
 0.17121588089330025]

### Percentage high PK

In [115]:
control_data = pd.read_csv(source_data + "full_survey_control_clean.csv", index_col = 0, low_memory=False)
control_data['pol_knowledge'] = control_data['Q_PK_1'].isin(["Democratic Party"])*1 + control_data['Q_PK_3'].isin(["Nancy Pelosi"])*1 + control_data['Q_PK_6'].isin(["Prime Minister of the United Kingdom"])*1 + control_data['Q_PK_7'].isin(["Michael Pompeo"])*1


control_data_noncovid = pd.read_csv(source_data + "full_survey_noncovid_control_clean.csv", index_col = 0, low_memory=False)
control_data_noncovid['pol_knowledge'] = control_data_noncovid['Q_PK_1'].isin(["Democratic Party"])*1 + control_data_noncovid['Q_PK_3'].isin(["Nancy Pelosi"])*1 + control_data_noncovid['Q_PK_6'].isin(["Prime Minister of the United Kingdom"])*1 + control_data_noncovid['Q_PK_7'].isin(["Michael Pompeo"])*1


control_data_covid = pd.read_csv(source_data + "full_survey_covid_control_clean.csv", index_col = 0, low_memory=False)
control_data_covid['pol_knowledge'] = control_data_covid['Q_PK_1'].isin(["Democratic Party"])*1 + control_data_covid['Q_PK_3'].isin(["Nancy Pelosi"])*1 + control_data_covid['Q_PK_6'].isin(["Prime Minister of the United Kingdom"])*1 + control_data_covid['Q_PK_7'].isin(["Michael Pompeo"])*1




In [116]:
#percentage all respondents with high political knowledge
((control_data['pol_knowledge'] == 4).sum() + (control_data_noncovid['pol_knowledge'] == 4).sum() + (control_data_covid['pol_knowledge'] == 4).sum())/(control_data.shape[0] + control_data_noncovid.shape[0] + control_data_covid.shape[0])

0.507315034136826

### Percentage high CRT

In [117]:
control_data['tot_crt'] = control_data['Q_1st_CRT'].str.contains("2", regex = False, case = False)*1 + control_data['Q_2nd_CRT'].isin(["7"])*1 + control_data['Q_3rd_CRT'].str.contains("emily", regex = False, case = False)*1 + control_data['Q_4th_CRT'].str.contains("0|no dirt|zero|none|nothing", regex = True, case = False)*1

In [118]:
control_data['tot_crt'].value_counts()/control_data.shape[0]

1    0.446767
2    0.335174
0    0.125000
3    0.089117
4    0.003352
Name: tot_crt, dtype: float64

In [119]:
len(article_dic_covid)

40

In [120]:
org_test_non = pickle.load(open(data_pickles + "test_set_noncovid.p", "rb"))
org_val_non = pickle.load(open(data_pickles + "val_set_noncovid.p", "rb"))
org_train_non = pickle.load(open(data_pickles + "train_set_noncovid.p", "rb"))

### creating article level table

In [96]:
article_df = pd.read_csv(prepared_data + "article_table_orig.csv")
all_sets_list_ext = all_sets_list.copy()
all_sets_list_ext.extend([1000])

In [97]:
new_arts = article_df[article_df.num.isin(all_sets_list_ext)]

In [98]:
new_arts.to_csv(prepared_data + "article_table_final.csv")